In [1]:
!pip install torch-snippets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp /content/drive/Shareddrives/MO434/dataset/project_dataset_corel_ct.zip .
!unzip -q -n project_dataset_corel_ct.zip

replace project_dataset_corel/fold0/.~lock.ct_val.csv#? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Exception in thread Thread-5 (attachment_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 237, in listen
    sock, _ = endpoints_listener.accept()
  File "/usr/lib/python3.10/socket.py", line 293, in accept
    fd, addr = self._accept()
TimeoutError: timed out

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 52, in attachment_entry
    debugpy.listen(_dap_port)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/public_api.py", line 31, in wrapper
    return wrapped(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/debugpy/server/api.py", line 143, in debug
    log.reraise

## Contrastive

In [4]:
from torch_snippets import *
import torch
import sys


batchsize = 32
n_fold = 3
input_shape = (3,224,224)
device = torch.device('cuda')
test_only = False

lr = 2e-5
weight_decay=1e-3
n_epochs = 60

scheduler_step=20
scheduler_gamma = 0.5

dataset_dir = "/content/project_dataset_corel"
#dataset_dir = "../dataset/project_dataset_corel"

log_dir='/content/drive/Shareddrives/MO434/executions/ct/ct_freeze'
#log_dir='./log'

def create_dir_if_necessary(dir):
    try:
        os.makedirs(dir)
    except FileExistsError:
        pass
create_dir_if_necessary(log_dir)

[07/05/23 23:59:42] WARNING  Unable to load torch and dependent libraries from                ]8;id=906588;file:///usr/local/lib/python3.10/dist-packages/torch_snippets/loader.py\loader.py]8;;\:]8;id=817743;file:///usr/local/lib/python3.10/dist-packages/torch_snippets/loader.py#<module>:98\<module>:98]8;;\
                             torch-snippets.                                                                       
                             Functionalities might be limited. pip install lovely-tensors in                       
                             case there are torch related errors                                                   

In [5]:
import pandas as pd
import typing
import os.path as osp
from PIL import Image

class CtDataset():
    def __init__(self, fold_dir, mode: typing.Literal['test', 'val', 'train'], transforms=None) -> None:
        self.fold_dir = fold_dir
        self.mode = mode
        self.transforms = transforms

        self.img_dir = osp.join(fold_dir, mode)
        self.pair_df = pd.read_csv(osp.join(fold_dir, f'ct_{self.mode}.csv'), header=None)

        self.img_dict = {}
        img_list = pd.concat((self.pair_df[0], self.pair_df[1]), ignore_index=True).drop_duplicates().reset_index(drop=True)

        for idx, item in img_list.items():
            img_path = osp.join(self.img_dir, item)
            img = Image.open(img_path)
            self.img_dict[item] = img

    def __len__(self):
        return len(self.pair_df)

    def __getitem__(self, idx):
        pair = self.pair_df.loc[idx, :]
        img1 = self.img_dict[pair[0]]
        img2 = self.img_dict[pair[1]]
        if self.transforms:
            img1 = self.transforms(img1)
            img2 = self.transforms(img2)
        return img1, img2, np.array([pair[2]])



In [6]:
from torchvision import transforms

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean

    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean

    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.10), scale=(0.9,1.1), shear=(-2,2),
                            interpolation=transforms.InterpolationMode.BILINEAR,
                            fill=0),
    transforms.Resize((224,224), interpolation=transforms.InterpolationMode.BILINEAR,
                      max_size=None, antialias=True),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    #AddGaussianNoise(0, 0.1)
])

val_transforms = transforms.Compose([
    transforms.Resize((224,224), interpolation=transforms.InterpolationMode.BILINEAR,
                      max_size=None, antialias=True),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [7]:
import torch
from torch import nn as nn
import math
from collections import OrderedDict


class CMKNetCt(nn.Module):
    def __init__(self, dropout=0) -> None:
        super().__init__()

        self.dropout = dropout

        # input: 3, 224, 224
        self.col1 = nn.Sequential(
            self._conv_seq(3, 32, 3), # 32, 112, 112
            self._conv_seq(32, 64, 3), # 64, 56, 56
            self._conv_seq(64, 128, 3), # 128, 28, 28
        )

        self.dense = nn.Sequential(
            self._linear_seq(128*28*28, 1024, dropout),
            self._linear_seq(1024, 256, dropout),
            nn.Linear(256, 64)
        )

        self._initialize_weights()

        self.cam = False
        self.cam_grad = None

    def forward(self, x):
        x = self.col1(x)
        x = torch.flatten(x, 1)
        x = self.dense(x)

        return x

    def _register_grad(self, grad):
        self.cam_grad = grad

    def get_conv_gradient(self):
        return self.cam_grad

    def get_conv_activation(self, x):
        return self.col1(x)

    def get_output_per_layer(self, x):
        output_by_layer = OrderedDict()

        output_by_layer['input'] = x.clone().detach().cpu().data.numpy()

        x = self.col1(x)
        output_by_layer["featextract-conv"] = x.clone().detach().cpu().numpy()

        x = torch.flatten(x, 1)
        for layer_name, layer in self.dense.named_children():
            #do forward through the layer
            x = layer.forward(x)
            #save the output
            output_by_layer["classifier-"+layer_name] = x.clone().detach().cpu().numpy()

        return output_by_layer


    def _fuse(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def _linear_seq(self, in_features, out_features, dropout):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def _conv_seq(self, in_channels, out_channels, kernel_size):
        "conv2d, batchnorm, relu, max pool"
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def _initialize_weights(self):
        #for each submodule of our network
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                #get the number of elements in the layer weights
                n = m.kernel_size[0] * m.kernel_size[1] * m.in_channels
                #initialize layer weights with random values generated from a normal
                #distribution with mean = 0 and std = sqrt(2. / n))
                m.weight.data.normal_(mean=0, std=math.sqrt(2. / n))

                if m.bias is not None:
                    #initialize bias with 0
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                #initialize layer weights with random values generated from a normal
                #distribution with mean = 0 and std = 1/100
                m.weight.data.normal_(mean=0, std=0.01)
                if m.bias is not None:
                #initialize bias with 0
                    m.bias.data.zero_()

def get_model_ct(params_dir=None):
    model = CMKNetCt()
    if params_dir:
        model.load_state_dict(torch.load(params_dir, map_location=device))
    return model

In [8]:
contrastive_thres = 1.1

class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin
    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2, keepdim = True)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2)/2 +
            (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))/2
        #print(list(zip(euclidean_distance.cpu().detach().numpy(), label.cpu().detach().numpy())))
        #print(loss_contrastive)
        acc = ((euclidean_distance > contrastive_thres) == label).float().mean()
        return loss_contrastive, acc

In [9]:
import matplotlib.pyplot as plt

def dist_hist(best_model_path, trn_dl):
    # Distance histograms for the cases we have the same and different persons, respectively.
    model = get_model_ct(best_model_path).to(device)
    different = []
    same      = []
    model.eval()

    for ix, data in enumerate(trn_dl):
        if (ix + 1)%100 == 0:
            print('Processing batch {}/{}'.format((ix + 1), len(trn_dl)))
        imgsA, imgsB, labels = [t.to(device) for t in data]
        codesA, codesB       = model(imgsA), model(imgsB)
        labels               = labels.cpu()

        with torch.no_grad():
            euclidean_distance        = F.pairwise_distance(codesA, codesB)
            euclidean_distance        = euclidean_distance.cpu()

        for i in range(len(labels)):
            if (labels[i] == 0): # same person
                same.append(euclidean_distance[i].item())
            else:
                different.append(euclidean_distance[i].item())

    fig, ax = plt.subplots(figsize = (10,5))
    ax.set_title('Distance histograms for genuine and impostor comparisons', fontsize = 20, fontweight = 'bold')
    ax.set_xlabel('Euclidean Distance', fontsize = 16, fontweight = 'bold')
    ax.set_ylabel('Density', fontsize = 16, fontweight = 'bold')
    ax.hist(different,bins = 50, density=True, alpha = 0.7, label = 'different')
    ax.hist(same, bins = 50, density=True, alpha = 0.7, label = 'same')
    ax.tick_params(labelsize = 16, axis = 'both')
    ax.legend()
    ax.grid(True)
    plt.plot()

def train_batch_ct(model, data, optimizer, criterion):
    imgsA, imgsB, labels = [t.to(device) for t in data]
    optimizer.zero_grad()
    codesA, codesB = model(imgsA), model(imgsB)
    loss, acc = criterion(codesA, codesB, labels)
    loss.backward()
    optimizer.step()
    return loss.item(), acc.item()

@torch.no_grad()
def validate_ct(model, dataloder, criterion):
    model.eval()
    ac_acc, ac_loss = 0, 0
    cnt = 0
    for idx, data in enumerate(dataloder):
        imgsA, imgsB, labels = [t.to(device) for t in data]
        codesA, codesB = model(imgsA), model(imgsB)
        loss, acc = criterion(codesA, codesB, labels)
        ac_loss += loss
        ac_acc += acc
        cnt +=1
    model.train()
    return ac_loss.item()/cnt, ac_acc.item()/cnt

In [10]:

def train_ct(dataset_dir, best_model_path):
    # DataLoaders
    train_set = CtDataset(dataset_dir, 'train', transforms=train_transforms)
    val_set = CtDataset(dataset_dir, 'val', transforms=val_transforms)

    trn_dl = DataLoader(train_set, batch_size=batchsize, shuffle=True)
    val_dl = DataLoader(val_set, batch_size=1, shuffle=False)

    model = get_model_ct().to(device)
    model.train()

    criterion = ContrastiveLoss()
    optimizer = optim.Adam(model.parameters(),lr = lr, weight_decay=weight_decay)

    log = Report(n_epochs)

    best_model = None
    best_model_loss = sys.maxsize

    for epoch in range(n_epochs):
        N = len(trn_dl)
        epoch_train_loss = 0
        for i, data in enumerate(trn_dl):
            loss, acc = train_batch_ct(model, data, optimizer, criterion)
            epoch_train_loss += loss

        loss, acc = validate_ct(model, val_dl, criterion)
        if loss < best_model_loss:
            torch.save(model.state_dict(), best_model_path)
            best_model_loss = loss
            best_model = epoch

        log.record(epoch+1, train_loss=epoch_train_loss/N, val_loss=loss, val_acc=acc,
        best_model=best_model+1, best_loss=best_model_loss, end='\r')

    log.plot_epochs(['train_loss','val_loss'])
    log.plot_epochs(['val_acc'])
    del model
    torch.cuda.empty_cache()
    dist_hist(best_model_path, trn_dl)

In [11]:
for fold in range(n_fold):
    fold_log_dir = osp.join(log_dir, f'fold{fold}')
    create_dir_if_necessary(fold_log_dir)

    train_ct(osp.join(dataset_dir, f'fold{fold}'), osp.join(fold_log_dir, 'best_model_ct.pth'))


# necessario remover dropout

KeyboardInterrupt: ignored

## Classification

In [ ]:
batchsize = 32
n_class = 6
n_fold = 3
input_shape = (3,224,224)
device = torch.device('cuda')
test_only = False
freeze = True

lr = 2e-5
weight_decay=1e-4
n_epochs = 100

model_name = 'CMKNet'

scheduler_step=20
scheduler_gamma = 0.5

dataset_dir = "/content/project_dataset_corel"

log_dir='/content/drive/Shareddrives/MO434/executions/ct/ct_freeze'

def create_dir_if_necessary(dir):
    try:
        os.makedirs(dir)
    except FileExistsError:
        pass
create_dir_if_necessary(log_dir)

In [ ]:
import torch
from torch import nn as nn
import math
from collections import OrderedDict


class CMKNet(nn.Module):
    def __init__(self, dropout=0.5) -> None:
        super().__init__()

        self.dropout = dropout

        # input: 3, 224, 224
        self.col1 = nn.Sequential(
            self._conv_seq(3, 32, 3), # 32, 112, 112
            self._conv_seq(32, 64, 3), # 64, 56, 56
            self._conv_seq(64, 128, 3), # 128, 28, 28
        )

        #self.col2 = nn.Sequential(
        #    self._conv_seq(3, 24, 5), # 32, 112, 112
        #    self._conv_seq(24, 48, 5), # 64, 56, 56
        #    self._conv_seq(48, 96, 5), # 128, 28, 28
        #)

        #self.fuse = self._conv_seq(128+96, 32, 1) # 32, 14, 14
        #self.fuse = self._conv_seq(128, 32, 1) # 32, 14, 14

        self.classifier = nn.Sequential(
            self._linear_seq(128*28*28, 1024, dropout),
            self._linear_seq(1024, 256, dropout),
            nn.Linear(256, n_class)
        )

        self._initialize_weights()

        self.cam = False
        self.cam_grad = None

    def forward(self, x):
        x = self.col1(x)
        #x = self.fuse(torch.cat((x1, x2), 1))
        if self.cam:
            x.register_hook(self._register_grad)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _register_grad(self, grad):
        self.cam_grad = grad

    def get_conv_gradient(self):
        return self.cam_grad

    def get_conv_activation(self, x):
        return self.col1(x)

    def get_output_per_layer(self, x):
        output_by_layer = OrderedDict()

        output_by_layer['input'] = x.clone().detach().cpu().data.numpy()

        x = self.col1(x)
        output_by_layer["featextract-conv"] = x.clone().detach().cpu().numpy()

        x = torch.flatten(x, 1)
        for layer_name, layer in self.classifier.named_children():
            #do forward through the layer
            x = layer.forward(x)
            #save the output
            output_by_layer["classifier-"+layer_name] = x.clone().detach().cpu().numpy()

        return output_by_layer


    def _fuse(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def _linear_seq(self, in_features, out_features, dropout):
        return nn.Sequential(
            nn.Linear(in_features, out_features),
            nn.ReLU(),
            nn.Dropout(dropout)
        )

    def _conv_seq(self, in_channels, out_channels, kernel_size):
        "conv2d, batchnorm, relu, max pool"
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=kernel_size//2, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def _initialize_weights(self):
        #for each submodule of our network
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                #get the number of elements in the layer weights
                n = m.kernel_size[0] * m.kernel_size[1] * m.in_channels
                #initialize layer weights with random values generated from a normal
                #distribution with mean = 0 and std = sqrt(2. / n))
                m.weight.data.normal_(mean=0, std=math.sqrt(2. / n))

                if m.bias is not None:
                    #initialize bias with 0
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                #initialize layer weights with random values generated from a normal
                #distribution with mean = 0 and std = 1/100
                m.weight.data.normal_(mean=0, std=0.01)
                if m.bias is not None:
                #initialize bias with 0
                    m.bias.data.zero_()

def get_model(params_dir=None, ct_param_dir=None, freeze=False):
    class_model = CMKNet()
    if params_dir:
        class_model.load_state_dict(torch.load(params_dir, map_location=device))
    if ct_param_dir:
        ct_model = get_model_ct(params_dir=ct_param_dir)
        if freeze:
            for param in ct_model.parameters():
                param.requires_grad = False
        class_model.col1 = ct_model.col1
    return class_model

In [ ]:
import pandas as pd
import typing
import os.path as osp
from PIL import Image

def list_join_dir(dir):
    return sorted([osp.join(dir, f) for f in os.listdir(dir)])

class DataSet():
    def __init__(self, fold_dir, mode: typing.Literal['test', 'val', 'train'], transforms=None) -> None:
        self.fold_dir = fold_dir
        self.mode = mode
        self.transforms = transforms

        self.img_dir = osp.join(fold_dir, mode)
        self.pair_df = pd.read_csv(osp.join(fold_dir, f'ct_{self.mode}.csv'), header=None)

        self.img_list = []
        self.gt_list = []
        img_list = pd.concat((self.pair_df[0], self.pair_df[1]), ignore_index=True).drop_duplicates().reset_index(drop=True)

        for idx, item in img_list.items():
            img_path = osp.join(self.img_dir, item)
            img_basename = osp.basename(img_path)
            class_id = int(img_basename.split('.')[0].split('_')[0])

            img = Image.open(img_path)
            self.img_list.append(img)
            self.gt_list.append(class_id)

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img = self.img_list[idx]
        if self.transforms:
            img = self.transforms(img)
        return img, self.gt_list[idx]


In [ ]:
from torch.utils.data import DataLoader
from torchsummary import summary


def train_batch(model, data, optimizer, criterion, device):
    optimizer.zero_grad()
    ims, targets = data
    ims     = ims.to(device=device)
    targets = targets.to(device=device)
    preds   = model(ims)
    loss = criterion(preds, targets)
    loss.backward()
    optimizer.step()

    return loss.item()

@torch.no_grad()
def validate(model, loader, criterion, device):
    #loader batchsize should be 1 for validation
    model.eval()
    rights = 0
    errors = 0
    cnt = len(loader)
    loss = 0
    for _, data in enumerate(loader):
        img, label = data
        img = img.to(device)
        label = label.to(device)
        pred = model(img)
        if torch.argmax(pred).item() == label.item():
            rights += 1
        else:
            errors += 1
        loss += criterion(pred, label)
    model.train()
    return loss/cnt, rights/cnt # average loss and accuracy

def train(dataset_dir, log_dir, best_model_path, best_model_ct_path):
    # DataLoaders
    train_set = DataSet(dataset_dir, 'train', transforms=train_transforms)
    val_set = DataSet(dataset_dir, 'val', transforms=val_transforms)

    train_loader = DataLoader(train_set, batch_size=batchsize, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=1, shuffle=False)

    # Define Model
    model = get_model(ct_param_dir=best_model_ct_path, freeze=freeze).to(device)
    summary(model,input_shape)

    # Optimizer, scheduler and loss
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

    log      = Report(n_epochs)
    model.train()

    best_model = None
    best_model_loss = sys.maxsize

    for epoch in range(n_epochs):
        N = len(train_loader)
        epoch_train_loss = 0
        for bx, data in enumerate(train_loader):
            loss = train_batch(model, data, optimizer, criterion, device)
            epoch_train_loss += loss

        loss, acc = validate(model, val_loader, criterion, device)
        if loss < best_model_loss:
            torch.save(model.state_dict(), best_model_path)
            best_model_loss = loss
            best_model = epoch

        log.record(epoch+1, train_loss=epoch_train_loss/N, val_loss=loss, val_acc=acc,
                best_model=best_model+1, best_loss=best_model_loss, end='\r')

        lr_scheduler.step()

    log.plot_epochs(['train_loss','val_loss'])
    log.plot_epochs(['val_acc'])

In [ ]:
from sklearn import metrics
import numpy as np
import torch
import pandas as pd

@torch.no_grad()
def test_fold(model, loader, criterion, device):
    #loader batchsize should be 1 for validation
    model.eval()
    preds, gts = [], []
    cnt = len(loader)
    loss = 0
    for _, data in enumerate(loader):
        img, label = data
        img = img.to(device)
        label = label.to(device)
        pred = model(img)
        preds.append(torch.argmax(pred).item())
        gts.append(label.item())
        loss += criterion(pred, label)
    return loss/cnt, preds, gts

def calculate_metrics(preds, gts):
    preds, gts = np.array(preds), np.array(gts)
    gen_acc = metrics.accuracy_score(gts, preds)
    gen_cohen_kappa = metrics.cohen_kappa_score(preds, gts)

    acc_per_class = []
    cohen_per_class = []

    n_class = gts.max() + 1

    for class_id in range(n_class):
        cohen_score = metrics.cohen_kappa_score(preds==class_id, gts==class_id)
        cohen_per_class.append(cohen_score)
        acc_score = metrics.accuracy_score(preds==class_id, gts==class_id)
        acc_per_class.append(acc_score)

    per_class_results = pd.DataFrame({
        'class': list(range(n_class)),
        'acc': acc_per_class,
        'cohen_kappa': cohen_per_class
    })
    return gen_acc, gen_cohen_kappa, per_class_results

def test(dataset_dir, log_dir, best_model_path):
    test_set = DataSet(dataset_dir, 'test', transforms=val_transforms)
    test_loader  = DataLoader(test_set, batch_size=1, shuffle=False)

    model = get_model(params_dir=best_model_path)
    model.to(device)

    criterion = nn.CrossEntropyLoss()

    loss, preds, gts = test_fold(model, test_loader, criterion, device)
    gen_acc, gen_cohen_kappa, per_class_r = calculate_metrics(preds, gts)

    result = f'Results \nAcc: {gen_acc:.4f}. cohen kappa: {gen_cohen_kappa:.4f}. Loss: {loss:.6f}'
    print(result)

    per_class_r.to_csv(osp.join(log_dir, 'per_class_result.csv'), index=False)

    with open(osp.join(log_dir, 'results.txt'), 'w') as f:
        f.write(result)

    return gen_acc, gen_cohen_kappa, per_class_r


### Train Test Folds

In [ ]:
all_acc, all_cohen_kappa, all_per_class_r = [], [], pd.DataFrame()

for fold in range(n_fold):
    fold_dir = osp.join(dataset_dir, f'fold{fold}')
    fold_log_dir = osp.join(log_dir, f'fold{fold}')
    create_dir_if_necessary(fold_log_dir)

    best_model_path = osp.join(fold_log_dir, 'best_model.pth')
    best_model_ct_path = osp.join(fold_log_dir, 'best_model_ct.pth')

    if not test_only:
        train(fold_dir, fold_log_dir, best_model_path, best_model_ct_path)
    gen_acc, gen_cohen_kappa, per_class_r = test(fold_dir, fold_log_dir, best_model_path)

    all_acc.append(gen_acc)
    all_cohen_kappa.append(gen_cohen_kappa)
    all_per_class_r = pd.concat((all_per_class_r, per_class_r), ignore_index=True)

In [ ]:
# 'class', 'acc', 'cohen_kappa'
all_per_class_r = all_per_class_r.groupby(by='class').agg(
    class_id=('class', 'last'),
    mean_acc=('acc', 'mean'),
    std_acc=('acc','std'),
    mean_kappa=('cohen_kappa', 'mean'),
    std_kappa=('cohen_kappa','std')
).reset_index(drop=True)

all_per_class_r.to_csv(osp.join(log_dir, 'per_class_result.csv'), index=False)

result = f"""Mean acc: f{np.mean(all_acc):.4f}. Std acc: f{np.std(all_acc):.4f}
Mean kappa: f{np.mean(all_cohen_kappa):.4f}. Std kappa: f{np.std(all_cohen_kappa):.4f}
"""
with open(osp.join(log_dir, 'results.txt'), 'w') as f:
    f.write(result)
print(result)

### Visualization

#### Projections

In [ ]:
fold = 0
out_dir = osp.join(log_dir, f'fold{fold}')
cam_out_dir = osp.join(out_dir, 'cam')
create_dir_if_necessary(cam_out_dir)


best_model_dir = osp.join(out_dir, 'best_model.pth')
test_set = DataSet(osp.join(dataset_dir, f'fold{fold}', 'test'), transforms=val_transforms)
raw_test_set = DataSet(osp.join(dataset_dir, f'fold{fold}', 'test'))
test_loader = DataLoader(test_set)

model = get_model(params_dir=best_model_dir)
model.to(device)
_ = model.eval()


In [ ]:
from collections import OrderedDict
import numpy as np

#get the outputs, and labels
def get_outputs(model, dataload, device):
    outputs_by_layer = None
    all_labels = None

    #get a batch from the dataload
    for inputs, labels in dataload:
        #move inputs to the correct device
        inputs = inputs.to(device)
        labels = labels.clone().detach().cpu().numpy()

        #get the activations for visualization
        outputs = model.get_output_per_layer(inputs)

        #save the outputs
        if outputs_by_layer is None:
            outputs_by_layer = outputs
            all_labels       = labels
        else:
            for layer in outputs:
                outputs_by_layer[layer] = np.concatenate((outputs_by_layer[layer], outputs[layer]), axis=0)
            all_labels = np.concatenate((all_labels, labels))

    return outputs_by_layer, all_labels

#maps from high dimension to 2D
def projection(outputs_by_layer, reducer):
    projection_by_layer = OrderedDict()

    for layer in outputs_by_layer:
        #get the output of layer
        output = outputs_by_layer[layer]
        output = output.reshape(output.shape[0], -1)
        #map to 2D
        embedded = reducer.fit_transform(output)
        #save projection
        projection_by_layer[layer] = embedded

    return projection_by_layer

#plot the projection of the output of each layer
def create_visualization(projection_by_layer, all_labels, out_dir):

    for layer in projection_by_layer:
        embedded = projection_by_layer[layer]

        fig = plt.figure(figsize=(8, 8))
        plt.scatter(embedded[:, 0], embedded[:, 1], c=all_labels, cmap=plt.get_cmap('tab10'))
        plt.axis("off")
        plt.title(layer)
        plt.colorbar()
        plt.tight_layout()
        plt.savefig(osp.join(out_dir, layer)+'.pdf')
        plt.show()
        plt.close(fig)

In [ ]:
from sklearn.manifold import TSNE

#reducer = umap.UMAP()
reducer = TSNE(perplexity=30)


outputs_by_layer, all_labels = get_outputs(model, test_loader, device)
projection_by_layer = projection(outputs_by_layer, reducer)

prejection_out_dir = osp.join(out_dir, 'tsne')
create_dir_if_necessary(prejection_out_dir)
create_visualization(projection_by_layer, all_labels, prejection_out_dir)